# Stepping stone:  fork and build GITenberg repos

Need to step back to define terms carefully -- lots of tokens:

need to pass in github username, password

need to pass in or generate:

* repo token, which gets encrypted
* travis token to use 


[The Travis CI Blog: Token, Token, Token](https://blog.travis-ci.com/2013-01-28-token-token-token/)

* you can often use a personal GitHub access token ([Personal Access Tokens](https://github.com/settings/tokens)) to do a lot of work....but to create other tokens, you need the GitHub password itself.

[Authorizations | GitHub Developer Guide](https://developer.github.com/v3/oauth_authorizations/):

> You can use this API to manage your OAuth applications. You can only access this API via Basic Authentication using your username and password, not tokens.

In [ ]:
from github3 import login
from github_settings import (ry_username, ry_password,
                             username, password,
                             token, 
                             GITENBERG_TRAVIS_TOKEN,
                             RDHYEE_TRAVIS_TOKEN, 
                             RDHYEE_DON_QUIXOTE_TOKEN)

from travispy import TravisPy


from itertools import islice
import requests


import base64
import datetime
import time


from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.backends import default_backend

from gitenberg_utils import GitenbergJob


class GitenbergTravisJob(GitenbergJob):
    def __init__(self, username, password, repo_name, repo_owner,
               update_travis_commit_msg,
               tag_commit_message, travis_token=None, repo_token=None):
        
        super(GitenbergTravisJob, self).__init__(username, password, repo_name, repo_owner,
               update_travis_commit_msg,
               tag_commit_message)
        
        self.username = username
        self.password = password
        
        self._travis_token = travis_token
        if travis_token is None:
            self._travis_token = self.travis_token()
            
        self._repo_token = repo_token    
        self._travis_repo_public_key = None
 
        self.travis = TravisPy.github_auth(self.travis_token())
        if self.gh_repo is not None:
            self.travis_repo = self.travis.repo(self.repo_slug)
       
        
    def public_key_for_travis_repo(self):
        if self._travis_repo_public_key is None:
            self._travis_repo_public_key =  requests.get("https://api.travis-ci.org/repos/{}/{}/key".format(self.repo_owner,
                                        self.repo_name)).json()['key']
        return self._travis_repo_public_key


    def travis_token(self):

        if self._travis_token is not None:
            return self._travis_token
        
        token_note = "token for travis {}".format(datetime.datetime.utcnow().isoformat())
        token = self.gh.authorize(self.username, self.password, 
                             scopes=('read:org', 'user:email', 'repo_deployment', 
                                     'repo:status', 'write:repo_hook'), note=token_note)

        return token.token
    
    def repo_token(self, from_repo_owner='GITenberg', create_duplicate=False):
        """
       
        """

        if self._repo_token is not None:
            return self._repo_token
        
        token_note = "automatic releases for {}/{}".format(self.repo_owner, self.repo_name)

        try:
            token = self.gh.authorize(username, password, scopes=('public_repo'), note=token_note)
        except Exception as e:
            if self._authorization_description_already_exists(e):
                # try again with a new description
                if create_duplicate:
                    token_note += " " + datetime.datetime.utcnow().isoformat()
                    token = self.gh.authorize(username, password, scopes=('public_repo'), note=token_note)
                else:
                    raise Exception('repo token already exists for {}'.format(self.repo_slug))
            else:
                raise e

        self._repo_token = token.token
        return self._repo_token
    
    def travis_encrypt(self, token_to_encrypt):
        """
        return encrypted version of token_to_encrypt 
        """
        
        # token_to_encrypt has to be string
        # if's not, assume it's unicode and enconde in utf-8
        
        if isinstance(token_to_encrypt, unicode):
            token_string = token_to_encrypt.encode('utf-8')
        else:
            token_string = token_to_encrypt
        
        repo_public_key_text = self.public_key_for_travis_repo() 
        repo_public_key = serialization.load_pem_public_key(repo_public_key_text.encode('utf-8'),
                                                            backend=default_backend())

        ciphertext = repo_public_key.encrypt(
         token_string,
         padding.PKCS1v15()
        )

        return base64.b64encode(ciphertext)
    
    
    @staticmethod
    def _authorization_description_already_exists(e):
        """
        Given an exception e when trying to create a token, is the exception the result of a duplicate description
        """
        if (e.code == 422 and 
            e.message == u'Validation Failed' and 
            (u'already_exists', u'description') in [(error['code'], error['field']) for error in e.errors]):
            return True
        else:
            return False

class ForkBuildRepo(GitenbergTravisJob):
    def fork_and_build_gitenberg_repo(self, from_repo_owner='GITenberg', 
                                      create_duplicate_token=False,
                                      update_repo_token_file=True):
        """

        """

        from_repo = self.gh.repository(from_repo_owner, self.repo_name)
        
        # fork if necessary
        if self.gh_repo is None:
            self.gh_repo = from_repo.create_fork()
            # instantiate self.travis_repo
            self.travis.user().sync()
            
            self.travis_repo = self.travis.repo(self.repo_slug)
           
        
        # make sure it's active
        if not self.travis_repo.enable():
            raise Exception("unable to enable travis repo:{}".format(self.repo_slug))
    
        encrypted_key = self.travis_encrypt(self.repo_token())
        
        # update .travis.deploy.api_key.txt if requested
        if update_repo_token_file:
            self.create_or_update_file(
                 path = ".travis.deploy.api_key.txt", 
                 message = "update .travis.deploy.api_key.txt with new encrypted token",
                 content = encrypted_key,
                 ci_skip = True)
            
        #  update .travis.yml
        
        self.create_or_update_file(
            path = ".travis.yml",
            message = "update .travis.yml with new token and repo_owner",
            content = self.update_travis_template(write_changes=False, 
                                encrypted_key=encrypted_key)[0].encode('utf-8'),
            ci_skip = True
        )

        # update version and tag commit -- should fire off a travis build
        tag_result = self.tag_commit(version_type='patch', write_changes=True)
        return tag_result
        


# rest of second folio

In [ ]:
from second_folio import (all_repos)

all_repos[:5]

In [ ]:
class BuildRepo(GitenbergTravisJob):
        
    def run(self, from_repo_owner='GITenberg', 
                                      create_duplicate_token=False,
                                      update_repo_token_file=True,
                                      load_repo_token=True):
        """

        """

        
        # make sure it's active
        if not self.travis_repo.enable():
            raise Exception("unable to enable travis repo:{}".format(self.repo_slug))
    
        if load_repo_token:
            try:
                encrypted_key = self.gh_repo.contents(".travis.deploy.api_key.txt").decoded
            except:
                encrypted_key = self.travis_encrypt(self.repo_token())
        else:
            encrypted_key = self.travis_encrypt(self.repo_token())
        
        # update .travis.deploy.api_key.txt if requested
        if update_repo_token_file:
            self.create_or_update_file(
                 path = ".travis.deploy.api_key.txt", 
                 message = "update .travis.deploy.api_key.txt with new encrypted token",
                 content = encrypted_key,
                 ci_skip = True)
            
        #  update .travis.yml
        
        self.create_or_update_file(
            path = ".travis.yml",
            message = "update .travis.yml with new token and repo_owner",
            content = self.update_travis_template(write_changes=False, 
                                encrypted_key=encrypted_key)[0].encode('utf-8'),
            ci_skip = True
        )

        # update version and tag commit -- should fire off a travis build
        tag_result = self.tag_commit(version_type='patch', write_changes=True)
        return tag_result
    
class BuildRepo2(BuildRepo):
    def next_version(self, version_type='patch'):
        """
        can be overriden -- by default next patch
        """

        assert version_type in ('patch', 'minor', 'major')

        import semantic_version
        
        _version = self.version()
        if _version < semantic_version.Version('0.1.0'):
            return semantic_version.Version('0.1.0')
        else:
            next_func = getattr(_version, "next_{}".format(version_type))
            return next_func()

In [ ]:
bj = BuildRepo(username=username, password=password, repo_name='Bleak-House_1023',
              repo_owner='GITenberg', 
              update_travis_commit_msg='update travis',
              tag_commit_message='update travis',
              travis_token=GITENBERG_TRAVIS_TOKEN)

In [ ]:
bj.next_version()

In [ ]:
bj.run()

In [ ]:
repos_to_build = [
    'Notre-Dame-De-Paris_2610',
    'Ulysses_4300',
    'Great-Expectations_1400',
    'The-Good-Soldier_2775'
]

In [ ]:
repos2_to_build = [
    'Great-Expectations_1400',
    'The-Good-Soldier_2775'
]

In [ ]:
repos_to_build[0:1]

In [ ]:
for repo in repos2_to_build[:]:
    try:
        bj = BuildRepo2(username=username, password=password, repo_name=repo,
              repo_owner='GITenberg', 
              update_travis_commit_msg='update travis',
              tag_commit_message='update travis',
              travis_token=GITENBERG_TRAVIS_TOKEN)
    
        bj.run()
    except Exception as e:
        print (repo, e)

In [ ]:
from gitenberg import metadata
import os

def local_yaml_file(id_):
    
    fname = "/Users/raymondyee/C/src/gitenberg-dev/giten_site/metadata/{}.yaml".format(id_)
    if os.path.exists(fname):
        md = metadata.pandata.Pandata(fname)
        return md
    else:
        return None
    


In [ ]:
for repo in all_repos[50:60]:
    id_ = repo.split("_")[-1]
    print repo, local_yaml_file(id_)

# changes to make in metadata file to initialize it

* _version 0.0.1

```
covers:							      
- attribution: Ed Gaither - Modern Electrographic, 2015	      
  cover_type: original					      
  image_path: cover.jpg					      
  rights: Attribution-NonCommercial 4.0 International (CC BY- 
  rights_url: https://creativecommons.org/licenses/by-nc/4.0/ 
```

```
publication_date 
publisher: Recovering the Classics	
rights: CC BY-NC
rights_url http://creativecommons.org/licenses/by-nc/4.0/
```


* add GITenberg subject

In [ ]:
gj.travis_encrypt(RDHYEE_DON_QUIXOTE_TOKEN)

In [ ]:
u = gj.gh.user()
u.email

In [ ]:
gj.fork_and_build_gitenberg_repo()

In [ ]:
gj.create_or_update_file(path='JUNK.md', message=b'updated junk.md', content=u'hello'.encode('utf-8'))

In [ ]:
print(gj.update_travis_template(write_changes=False, 
                                encrypted_key=gj.travis_encrypt(gj.repo_token()))[0])

## next step

* make sure travis_repo is active
* check on existence of metadata.yaml in the the new github repo
* write .travis.deploy.api_key.txt to the github repo
* compute new .travis.yml and write to github repo
* tag repo 

In [ ]:
from travispy import TravisPy


travis = TravisPy.github_auth(RDHYEE_TRAVIS_TOKEN)

t_user = travis.user()
t_user.login

In [ ]:
travis_repo = travis.repo('rdhyee/Don-Quixote_996')
travis_repo.active

travis_repo.enable()

In [ ]:
travis_encrypt(token_to_encrypt=token.token.encode('utf-8'),
               repo_slug="rdhyee/Don-Quixote_996")

# travispy

```
pip install travispy
```

[menegazzo/travispy: Travis CI API for Python](https://github.com/menegazzo/travispy)


In [ ]:
# let's create Github tokens for use with travispy

"""
read:org
user:email
repo_deployment
repo:status
write:repo_hook
"""

from github3 import (login)
import datetime


def create_github_token_for_travis(username, password):
    gh = login(username, password)
    
    token_note = "token for travis {}".format(datetime.datetime.utcnow().isoformat())
    token = gh.authorize(username, password, 
                         scopes=('read:org', 'user:email', 'repo_deployment', 
                                 'repo:status', 'write:repo_hook'), note=token_note)
            
    return token
    

In [ ]:
travis_token = create_github_token_for_travis(ry_username, ry_password)
travis_token

In [ ]:
travis_token.token

In [ ]:
from travispy import TravisPy
from github_settings import (ry_username, ry_password, RDHYEE_TRAVIS_TOKEN)

# travis_token = create_github_token_for_travis(ry_username, ry_password)

t = TravisPy.github_auth(RDHYEE_TRAVIS_TOKEN)

t_user = t.user()
t_user

# create test parameters for travispy

https://github.com/menegazzo/travispy/blob/v0.3.4/travispy/_tests/test_settings.example.json

In [ ]:
import json
from travispy import TravisPy

def create_travispy_test_settings(github_token, repo_slug):
    settings = {}
    
    travis = TravisPy.github_auth(github_token)
    
    settings['github_token'] = github_token
    settings['repo_slug'] = repo_slug
    
    # account
    # https://github.com/menegazzo/travispy/blob/v0.3.4/travispy/_tests/test_authenticated.py#L31
    
    accounts = travis.accounts()
    account = travis.account(accounts[0].id)
    
    settings['account'] = {
        'count': len(accounts),
        'id': account.id,
        'name': account.name,
        'login': account.login,
        'type': account.type,
        'repos_count': account.repos_count,
        'subscribed': hasattr(account, 'subscribed')
    }
    
    # hook
    # https://github.com/menegazzo/travispy/blob/v0.3.4/travispy/_tests/test_authenticated.py#L73
    
    hooks = travis.hooks()
    hook = hooks[0]

    settings['hook'] = {
        'count': len(hooks),
        'name': hook.name,
        'description': hook.description,
        'owner_name': hook.owner_name,
        'active': hook.active,
        'private': hook.private,
        'admin': hook.admin
    }
    
    # user
    # https://github.com/menegazzo/travispy/blob/v0.3.4/travispy/_tests/test_authenticated.py#L110
    
    user = travis.user()

    settings['user'] = {
        'login': user['login'],
        'name': user['name']
    }
    
    # branch
    # https://github.com/menegazzo/travispy/blob/v0.3.4/travispy/_tests/test_not_authenticated.py#L19
    branches = travis.branches(slug=repo_slug)
    branch = travis.branch('master', repo_slug)
    
    settings['branch'] = {
        'count': len(branches),
        'id': branch.id,
        'repository_id': branch.repository_id,
        'pull_request': branch.pull_request,
        'config': branch.config,
        'number': branch.number
    }
    
    # build
    # https://github.com/menegazzo/travispy/blob/v0.3.4/travispy/_tests/test_not_authenticated.py#L66

    builds = travis.builds(slug=repo_slug)
    build = travis.build(builds[0].id)
    build_id = builds[0].id
    
    settings['build'] = {
        'count': len(builds),
        'id': build.id,
        'repository_id': build.repository_id,
        'number': build.number,
        'pull_request': build.pull_request,
        'pull_request_title': build.pull_request_title,
        'pull_request_number': build.pull_request_number,
        'config': build.config
    }
    
    # commit
    # https://github.com/menegazzo/travispy/blob/v0.3.4/travispy/_tests/test_not_authenticated.py#L115
    
    commit = build.commit
    
    settings['commit'] = {
        'count': len(builds),
        'id': commit.id,
        'sha': commit.sha,
        'branch': commit.branch,
        'message': commit.message,
        'committed_at': commit.committed_at,
        'author_name': commit.author_name,
        'author_email': commit.author_email,
        'commiter_name': commit.committer_name, # sic
        'commiter_email': commit.committer_email, # sic
        'compare_url': commit.compare_url,
        'pull_request_number': None if not hasattr(commit, 'pull_request_number') else commit.pull_request_number
    }


    # jobs
    # https://github.com/menegazzo/travispy/blob/v0.3.4/travispy/_tests/test_not_authenticated.py#L140
    
 
    jobs = travis.jobs(ids=build.job_ids)
    job = travis.job(build.job_ids[0])
    
    settings['job'] = {
        'count': len(jobs),
        'build_id': job.build_id,
        'repository_id': job.repository_id,
        'number': job.number,
        'config': job.config,
        'queue': job.queue,
        'allow_failure': job.allow_failure,
        'annotation_ids': job.annotation_ids
        
    }
    
    # repo
    # https://github.com/menegazzo/travispy/blob/v0.3.4/travispy/_tests/test_not_authenticated.py#L252
    # let's add fake stuff for now
    
    settings['repo'] =  {
        "public_count": 25,
        "member_count": 5,
        "owner_count": 7,
        "github_language": "Python",
        "id": 2598876,
        "description": "TravisPy test project for Python 2.7",
        "active": True
  }

    
    return json.dumps(settings, indent=2)

In [ ]:
print(create_travispy_test_settings(RDHYEE_TRAVIS_TOKEN, 'rdhyee/hello-travis-ci'))